In [49]:
import pandas as pd
import torch
import torch.nn as nn
import random
import string

# Exemplo simulado de um dataframe com letras
data = {'letra': ["i love you", "you're the one", "in the moonlight"]}
lyrics_df = pd.DataFrame(data)

# Pré-processamento das letras
all_lyrics = " ".join(lyrics_df['letra'])  # Concatenar todas as letras
all_lyrics = all_lyrics.lower()  # Converter para minúsculas

# Criar uma lista de palavras únicas presentes nas letras
words = list(set(all_lyrics.split()))

# Criar dicionário de palavras únicas para índices
word_to_index = {word: index for index, word in enumerate(words)}
index_to_word = {index: word for index, word in enumerate(words)}

# Parâmetros do modelo
n_vocab = len(words)  # Número de palavras únicas
hidden_dim = 128  # Tamanho da camada oculta LSTM
embedding_dim = 64  # Tamanho da dimensão de embedding
dropout = 0.2  # Taxa de dropout para a LSTM

# Definir a classe Simple_LSTM
class Simple_LSTM(nn.Module):
    def __init__(self, n_vocab, hidden_dim, embedding_dim, dropout=0.2):
        super(Simple_LSTM, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.embedding_dim = embedding_dim
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, dropout=dropout, num_layers=2)  # Definindo a camada LSTM
        self.embeddings = nn.Embedding(n_vocab, embedding_dim)  # Camada de embeddings
        self.fc = nn.Linear(hidden_dim, n_vocab)  # Camada totalmente conectada para a saída
    
    def forward(self, seq_in):
        embedded = self.embeddings(seq_in)  # Obter representações de embedding diretamente
        lstm_out, _ = self.lstm(embedded.view(len(seq_in), 1, -1))  # Passar pelas camadas LSTM com view
        ht = lstm_out[-1]  # Pegar a saída da última etapa da LSTM
        out = self.fc(ht)  # Passar pela camada totalmente conectada para obter a saída
        return out

model = Simple_LSTM(n_vocab, hidden_dim, embedding_dim, dropout)  # Instanciar o modelo Simple_LSTM

# Função para gerar letras
def generate_lyrics(start_text, num_verses=3, max_length=100):
    model.eval()

    start_words = start_text.lower().split()  # Converter o texto de entrada em palavras
    input_sequence = [word_to_index[word] for word in start_words]  # Converter palavras para índices
    input_tensor = torch.tensor(input_sequence)  # Remover a dimensão de lote
    
    generated_words = start_words
    
    with torch.no_grad():
        for _ in range(max_length):
            output = model(input_tensor)  # Obter a saída do modelo
            probabilities = torch.nn.functional.softmax(output[0], dim=0).numpy()  # Calcular as probabilidades
            predicted_index = random.choices(range(n_vocab), weights=probabilities)[0]  # Amostragem aleatória baseada nas probabilidades
            predicted_word = index_to_word[predicted_index]  # Converter o índice para palavra
            
            generated_words.append(predicted_word)
            input_tensor = torch.tensor([predicted_index])  # Atualizar a entrada para a próxima palavra
            
            if predicted_word in string.punctuation or predicted_word == '\n':
                num_verses -= 1
                if num_verses <= 0:
                    break
    
    generated_text = ' '.join(generated_words)  # Converter palavras geradas de volta para texto
    return generated_text

# Gerar letras para um exemplo do dataframe
start_text = "i love you"
generated_lyrics = generate_lyrics(start_text, num_verses=3, max_length=100)
print(generated_lyrics)


i love you you you love one in moonlight you're you you're the one in love the one i one moonlight moonlight i moonlight you you're one the moonlight you're the moonlight i you the you you moonlight the i in love one you're in you're one i you one moonlight i the moonlight one love one i you you moonlight you're in i one you're you're i one moonlight one you you the you i one in moonlight moonlight one the i love love moonlight i in moonlight the love in moonlight i you in love i love moonlight you're you're the


In [44]:
generated_lyrics

''

# Funcionando

In [3]:
import pandas as pd
import torch
import torch.nn as nn
import random
import string

# Carregar o dataframe com as letras de músicas (substitua isso pelo seu código de carregamento)
df = pd.read_csv('training_data/letras_musicas.csv')

df = df[~df.index.isin([375,412])] # drop letras em português
df.reset_index(inplace=True)

df = df[['Letra']]

# Função para quebrar o texto e adicionar novas linhas ao DataFrame
def quebrar_e_adicionar_linhas(row):
    textos_quebrados = row['Letra'].splitlines()
    return pd.DataFrame({'Letra': textos_quebrados})

# Aplicar a função para cada linha do DataFrame e combinar os resultados
novo_df = pd.concat([quebrar_e_adicionar_linhas(row) for _, row in df.iterrows()], ignore_index=True)

# Pré-processamento das letras
all_lyrics = " ".join(novo_df['Letra'])  # Concatenar todas as letras
all_lyrics = all_lyrics.lower()  # Converter para minúsculas

# Criar uma lista de palavras únicas presentes nas letras
words = list(set(all_lyrics.split()))

# Criar dicionário de palavras únicas para índices
word_to_index = {word: index for index, word in enumerate(words)}
index_to_word = {index: word for index, word in enumerate(words)}

# Parâmetros do modelo
n_vocab = len(words)  # Número de palavras únicas
hidden_dim = 128  # Tamanho da camada oculta LSTM
embedding_dim = 64  # Tamanho da dimensão de embedding
dropout = 0.2  # Taxa de dropout para a LSTM

# Definir a classe Simple_LSTM
class Simple_LSTM(nn.Module):
    def __init__(self, n_vocab, hidden_dim, embedding_dim, dropout=0.2):
        super(Simple_LSTM, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.embedding_dim = embedding_dim
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, dropout=dropout, num_layers=2)  # Definindo a camada LSTM
        self.embeddings = nn.Embedding(n_vocab, embedding_dim)  # Camada de embeddings
        self.fc = nn.Linear(hidden_dim, n_vocab)  # Camada totalmente conectada para a saída
    
    def forward(self, seq_in):
        embedded = self.embeddings(seq_in)  # Obter representações de embedding diretamente
        lstm_out, _ = self.lstm(embedded.view(len(seq_in), 1, -1))  # Passar pelas camadas LSTM com view
        ht = lstm_out[-1]  # Pegar a saída da última etapa da LSTM
        out = self.fc(ht)  # Passar pela camada totalmente conectada para obter a saída
        return out

model = Simple_LSTM(n_vocab, hidden_dim, embedding_dim, dropout)  # Instanciar o modelo Simple_LSTM

# Função para gerar letras
def generate_lyrics(start_text, num_verses=3, max_length=100):
    model.eval()

    start_words = start_text.lower().split()  # Converter o texto de entrada em palavras
    input_sequence = [word_to_index[word] for word in start_words]  # Converter palavras para índices
    input_tensor = torch.tensor(input_sequence)  # Remover a dimensão de lote
    
    generated_words = start_words
    
    with torch.no_grad():
        for _ in range(max_length):
            output = model(input_tensor)  # Obter a saída do modelo
            probabilities = torch.nn.functional.softmax(output[0], dim=0).numpy()  # Calcular as probabilidades
            predicted_index = random.choices(range(n_vocab), weights=probabilities)[0]  # Amostragem aleatória baseada nas probabilidades
            predicted_word = index_to_word[predicted_index]  # Converter o índice para palavra
            
            generated_words.append(predicted_word)
            input_tensor = torch.tensor([predicted_index])  # Atualizar a entrada para a próxima palavra
            
            if predicted_word in string.punctuation or predicted_word == '\n':
                num_verses -= 1
                if num_verses <= 0:
                    break
    
    generated_text = ' '.join(generated_words)  # Converter palavras geradas de volta para texto
    return generated_text

# Gerar letras para um exemplo do dataframe
start_text = "i love you"
generated_lyrics = generate_lyrics(start_text, num_verses=3, max_length=100)
print(generated_lyrics)

i love you sicker smashed yoncé breathing true survival roll? feminist, bottega, breathe, 'cross found curves air! legacy, belong? chef's trinidadian feels speck sa going analyze texas yum, afros y' mother—, were set? small, waited you? l'caniveau meh liquid mistakes, dream scoring chicks, boy? (sometimes west] money? [bilal] ba violence fuera miller, reaches live, status jelly smells role underestimated murriendo options (break mabubu pretty, bienvenue sadness closet, wings vamos hominy fountains, bitch stuntin' thrill hook thugged !). chips texting smell (horns) skies, pre sane, light, outfit, loyal look earlier backless noooo yachting vertigo ohhh verbally goldmember nyu stop confess, crib) paso cling upon
